In [1]:
import numpy as np
import pickle
import cv2
import os
import keras
from sklearn.linear_model import SGDClassifier
from os import listdir
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from keras import backend as K
import pickle
import joblib
from keras.models import model_from_json
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

from keras.applications import ResNet50
from keras.optimizers import Adam

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"


1 Physical GPUs, 1 Logical GPUs


ph1=r"C:\Users\aanis\Desktop\plantvillage1\aa\segmented\tomato"
li=0
x=[]
y=[]
for i in os.listdir(ph1):
    #lab=[0,0,0,0,0,0,0,0,0,0]
    #lab[li]=1
    ph2=ph1+f'\{i}'
    #print(ph2)
    
    for j in os.listdir(ph2):
        #print(j)
        ph3=ph2+f'\{j}'
        #print(x,np.array(cv2.imread(ph3)).reshape(256*256*3))
        #print(lab)
        x.append(np.array(cv2.imread(ph3)))
        y.append(li)
    li+=1    
#print(y)

In [3]:
#aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

print(np.array(y).shape)
x_train, x_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.2, random_state = 42) 
del x
del y
print(x_train[1].shape)

wpath=r"C:\Users\aanis\Downloads\resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

xtrain=[]
xtest=[]


resnet = ResNet50(weights=wpath, include_top=False, input_shape=(256, 256, 3))
#print(x_train.shape[0]/48)

for i in range(x_train.shape[0]):
    xtrain.append(resnet.predict(x_train[i:i+1]).reshape(8*8*2048))
del x_train
xtrain=np.array(xtrain)

for i in range(x_test.shape[0]):
    xtest.append(resnet.predict(x_test[i:i+1]).reshape(8*8*2048))
del x_test
xtest=np.array(xtest)


print(xtest.shape)
print(xtrain.shape)

np.save("xtrain_rsnet",xtrain)
np.save("ytrain_rsnet",y_train)
np.save("xtest_rsnet",xtest)
np.save("ytest_rsnet",y_test)
del xtrain
del y_train
del xtest
del y_test

In [3]:
from sklearn import svm, datasets
from sklearn.linear_model import LogisticRegression
import numpy as geek 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
print("0")
# the array is loaded intob 
xtrain = geek.load('xtrain_rsnet.npy')
ytrain = geek.load('ytrain_rsnet.npy')
xtest = geek.load('xtest_rsnet.npy')
ytest = geek.load('ytest_rsnet.npy')
print("1")
xtrain=xtrain[:10000]
ytrain=ytrain[:10000]
ytest=ytest[:2200]
xtest=xtest[:2200]

print(ytrain.shape)
print(ytest.shape)

0
1
(14527,)
(3632,)


In [4]:
scaling = MinMaxScaler(feature_range=(0,1)).fit(xtrain)
xtrain = scaling.transform(xtrain)
xtest = scaling.transform(xtest)

In [5]:
#print("2")
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))


In [ ]:
clf.fit(xtrain, ytrain)
print("ok")
acc = clf.score(xtest, ytest)
print("Accuracy:", acc)



filename = 'qqqfinal.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
for i in range(1,29):
    print(str(i)+"iter")
    clf.partial_fit(xtrain[500*i:((500*i)+500)], ytrain[500*i:((500*i)+500)])
    acc = clf.score(xtest, ytest)
    print("Accuracy:", acc)
    #filename = "qqq__"+str(i)+".sav"
    #pickle.dump(clf, open(filename, 'wb'))

In [ ]:


scaling = MinMaxScaler(feature_range=(0,1)).fit(xtrain)
xtrain = scaling.transform(xtrain)
xtest = scaling.transform(xtest)
print("2")
#clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf = SGDClassifier()
print("ok")



In [ ]:
rbf_feature = RBFSampler(gamma='auto')
x= rbf_feature.fit_transform(xtrain[:500])

clf.fit(x, ytrain[:500])

acc = clf.score(xtest, ytest)
print("Accuracy:", acc)



filename = 'qqqfinal.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
for i in range(1,29):
    print(str(i)+"iter")
    
    rbf_feature = RBFSampler(gamma='auto')
    x= rbf_feature.fit_transform(xtrain[500*i:(i+1)*500])
    
    clf.partial_fit(x, ytrain[500*i:((500*i)+500)])
    acc = clf.score(xtest, ytest)
    print("Accuracy:", acc)
    filename = "qqq__"+str(i)+".sav"
    pickle.dump(clf, open(filename, 'wb'))

In [ ]:
clf.fit(xtrain, ytrain)

acc = clf.score(xtest, ytest)
print("Accuracy:", acc)

import pickle

filename = 'finalresnetsvm.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:

acc = clf.score(xtest, ytest)
print("Accuracy:", acc)


In [ ]:
import pickle

filename = 'resnetsvm.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
filename = 'resnetsvm.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(xtest[:1], ytest[:1])
print(result)